## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-05-01-10-19 +0000,nypost,"White House touts arrests of more than 4,000 c...",https://nypost.com/2026/02/04/us-news/white-ho...
1,2026-02-05-01-05-16 +0000,nypost,"Boy, 9, severely burned after microwaving NeeD...",https://nypost.com/2026/02/04/us-news/illinois...
2,2026-02-05-01-04-31 +0000,nyt,Trump Says He Ordered Withdrawal of 700 Immigr...,https://www.nytimes.com/2026/02/04/us/politics...
3,2026-02-05-00-59-53 +0000,nyt,Supreme Court Clears Way for California Voting...,https://www.nytimes.com/2026/02/04/us/politics...
4,2026-02-05-00-56-40 +0000,latimes,L.A. Olympics head Casey Wasserman withstood a...,https://www.latimes.com/california/story/2026-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2462/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
18,trump,37
40,epstein,27
120,new,27
28,california,14
22,immigration,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
237,2026-02-04-16-39-42 +0000,nypost,New Epstein emails about top Goldman lawyer Ka...,https://nypost.com/2026/02/04/business/epstein...,87
56,2026-02-04-23-11-15 +0000,nypost,Trump gets help in his critical minerals race ...,https://nypost.com/2026/02/04/us-news/house-pa...,87
167,2026-02-04-19-57-31 +0000,nypost,President Trump called Savannah Guthrie to off...,https://nypost.com/2026/02/04/us-news/presiden...,87
4,2026-02-05-00-56-40 +0000,latimes,L.A. Olympics head Casey Wasserman withstood a...,https://www.latimes.com/california/story/2026-...,83
114,2026-02-04-21-43-54 +0000,nyt,Melinda French Gates Addresses New References ...,https://www.nytimes.com/2026/02/04/us/melinda-...,81


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
237,87,2026-02-04-16-39-42 +0000,nypost,New Epstein emails about top Goldman lawyer Ka...,https://nypost.com/2026/02/04/business/epstein...
56,87,2026-02-04-23-11-15 +0000,nypost,Trump gets help in his critical minerals race ...,https://nypost.com/2026/02/04/us-news/house-pa...
4,56,2026-02-05-00-56-40 +0000,latimes,L.A. Olympics head Casey Wasserman withstood a...,https://www.latimes.com/california/story/2026-...
30,53,2026-02-05-00-11-37 +0000,nypost,"ICE warns California, Gavin Newsom to end poli...",https://nypost.com/2026/02/04/us-news/ice-warn...
40,53,2026-02-04-23-48-00 +0000,wsj,Federal Reserve governor Lisa Cook sees a grea...,https://www.wsj.com/economy/central-banking/fe...
182,48,2026-02-04-19-24-56 +0000,startribune,Federal lawyer tells judge ‘job sucks’ amid ov...,https://www.startribune.com/federal-lawyer-tel...
268,42,2026-02-04-14-52-00 +0000,wsj,"America’s private sector added 22,000 jobs las...",https://www.wsj.com/economy/jobs/adp-numbers-s...
242,42,2026-02-04-16-21-17 +0000,nypost,Tempers flare between Dem Rep. Maxine Waters a...,https://nypost.com/2026/02/04/us-news/tempers-...
167,40,2026-02-04-19-57-31 +0000,nypost,President Trump called Savannah Guthrie to off...,https://nypost.com/2026/02/04/us-news/presiden...
91,40,2026-02-04-22-17-00 +0000,wsj,Washington Post Lays Off One-Third of Staff in...,https://www.wsj.com/business/media/washington-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
